# Merge all Config Manager features into one dataframe
### Step 1: Load data into pandas dataframes 
Our team has spent some time in creating features from the tables in the Microsoft Configuration Manager database. Our features are stored in separate sources for each Config Manager table that we created features from. We have two types of data sources of features that we need to join together for our model:
1. `.parquet` files in blob storage 
2. SQL views from our cloud database 

We have tracked the data source file names and SQL views using a spreadsheet, and will use this list to pull the filenames and data from our cloud database and blob storage.


In [1]:
# Import modules 
from azure.storage.blob import BlobServiceClient
import pandas as pd 
import numpy
import pyodbc
import os
import json
import time

# Function Definitions --------------

def download_blob(cred, LOCALFILENAME,
                  CONTAINERNAME, BLOBNAME):

    # Start tracking download time
    t1=time.time()

    # Instantiate a blob service instance
    blob_service_client_instance = BlobServiceClient(account_url=cred['in_the_sun'], credential=cred['fun'])
    blob_client_instance = blob_service_client_instance.get_blob_client(CONTAINERNAME, BLOBNAME, snapshot=None)

    # Create the file locally
    with open(LOCALFILENAME, "wb") as my_blob:
        blob_data = blob_client_instance.download_blob()
        blob_data.readinto(my_blob)
    
    # Notify user of download time 
    t2=time.time()
    print(("It takes %s seconds to download "+BLOBNAME) % (t2 - t1))

def connect_to_database(cred):
    ''' 
    Uses pyodbc to connect to a SQL Server database
    and returns the connection object
    '''
    # Define database driver, server, database
    driver = 'SQL SERVER'

    # Define the connection string
    conn_str = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={cred['server']};DATABASE={cred['db']};UID={cred['uid']};PWD={cred['pwd']}"

    # Connect to database using windows authentication 
    conn = pyodbc.connect(conn_str)

    return conn 

def get_sql_data(persist_conn_obj, sql_query):
    ''' 
    Gets rows from a table in the SQL database connection
    '''
    # Execute a query for table table
    df_pers_all = pd.read_sql_query(sql_query, persist_conn_obj)
    
    return df_pers_all

def get_creds(in_file):
    with open(in_file) as f:
        cred = json.load(f)
    return cred

# Get creds 
creds_file = 'assets/fun.json'
creds = get_creds(creds_file)


In [66]:
def get_feature_metadata():

    # Read the EDA feature metadata 
    df = pd.read_excel('assets/EDA _ Feature Engineering Tasks.xlsx', sheet_name='Features to Engineer')

    # Select only completed features 
    df['Completed? (Y/N)'] = df['Completed? (Y/N)'].apply(lambda x: x.strip())
    df = df[df['Completed? (Y/N)'] == 'Y']

    # Get only one row per view/filename
    out_df = df[['Completed dataset location', 'View/Filename', 'Keys']]
    out_df = out_df.drop_duplicates()

    return out_df

def download_all_data(metadata_df, save_directory, creds):

    # Instantiate a dictionary to accumulate dataframes
    feature_dfs = {}

    # Start a SQL connection 
    conn = connect_to_database(creds)

    # Iterate through each view/file
    for row in metadata_df.iterrows():
        
        row = row[1]

        # Download and read Blob file
        if row['Completed dataset location'] == 'Blob Storage':
            
            # Download from blob storage
            blob_name = row['View/Filename']
            file_save_name = os.path.join(save_directory, blob_name)
            download_blob(creds, file_save_name, 'configmanagertest1', blob_name)

            # Read the blob into a dataframe and add to output dict
            current_df = pd.read_parquet(file_save_name)
            feature_dfs[blob_name] = current_df

        # Download and read SQL 
        if row['Completed dataset location'] == 'SQL View':
            
            # Call SQL to get data
            view_name = row['View/Filename']
            sql_query = f'SELECT * FROM {view_name}'
            current_df = get_sql_data(conn, sql_query)

            # Add data to output dict
            feature_dfs[view_name] = current_df
            print(f'retrieved {view_name} from SQL')
    
    return feature_dfs

# Download data 
save_location = 'assets/features'
df = get_feature_metadata()
feature_datasets = download_all_data(df, save_location, creds)


It takes 0.5489280223846436 seconds to download add_remove_programs_features.parquet
It takes 0.3425319194793701 seconds to download office_addin_features.parquet


/var/folders/_m/rvvvdp755xg8_b7k34mkcsvh0000gn/T/ipykernel_2525/2173581404.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pers_all = pd.read_sql_query(sql_query, persist_conn_obj)


retrieved Machine_Enclosure_Types from SQL
It takes 0.18597984313964844 seconds to download update_summary_features.parquet


/var/folders/_m/rvvvdp755xg8_b7k34mkcsvh0000gn/T/ipykernel_2525/2173581404.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pers_all = pd.read_sql_query(sql_query, persist_conn_obj)


retrieved Battery_Health_View from SQL
It takes 0.2097632884979248 seconds to download browser_usage_features.parquet
It takes 0.1256399154663086 seconds to download computer_system_features.parquet


/var/folders/_m/rvvvdp755xg8_b7k34mkcsvh0000gn/T/ipykernel_2525/2173581404.py:52: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pers_all = pd.read_sql_query(sql_query, persist_conn_obj)


retrieved Logical_Disk_Data_View from SQL
retrieved Machine_Office_Is32Bit from SQL
retrieved Office_Readiness_View from SQL
retrieved Processor_DATA_View from SQL
It takes 0.3371257781982422 seconds to download user_count_feature.parquet
It takes 0.5912771224975586 seconds to download boot_events_parsed_features.pq
It takes 0.4516150951385498 seconds to download num_events.pq
It takes 0.7315959930419922 seconds to download operating_system_features.pq
It takes 0.18439698219299316 seconds to download pc_bios_data.pq
It takes 0.14695191383361816 seconds to download pc_memory_data.pq
It takes 0.6722471714019775 seconds to download system_disc_features.pq
It takes 1.3645339012145996 seconds to download workstationstatus_features.pq


In [53]:
def view_data(in_dict):

    for key in in_dict.keys():

        dat = in_dict[key]
        print(key)
        yield dat

generator = view_data(feature_datasets)

In [58]:
current = next(generator)
current.head()

num_events.pq


,RWB_EFFECTIVE_DATE,ClientItemKey,events
0,2022-08-30,16783564,11
1,2022-08-30,16783835,22
2,2022-08-30,16784042,32
3,2022-08-30,16784339,5
4,2022-08-30,16784657,5


### Step 2: Join all features into one dataframe 


In [70]:
def join_all_features(features_dict, metadata_df):

    # Get the system_disc table 
    system_name = 'system_disc_features.pq'
    system_df = features_dict[system_name]

    # Iterate through each file 
    for row in metadata_df.iterrows():
        row = row[1]

        # Get the table and key on the right to join to system disc
        right_df_name = row['View/Filename']
        right_df = features_dict[right_df_name]
        right_keys = row['Keys'].split(', ')

        # skip if df is system disc
        if right_df_name == system_name:
            continue
        
        # Get the system disc key
        if ('RWB_EFFECTIVE_DATE' in right_keys) or ('created_date' in right_keys):
            left_keys = ['ItemKey', 'RWB_EFFECTIVE_DATE']
        else:
            left_keys = ['ItemKey']

        # Perform a left join for the features 
        system_df = pd.merge(system_df, right_df, 
                             how='left',
                             left_on=left_keys,
                             right_on=right_keys)
        
        if 'MachineID' in system_df.columns:
            system_df = system_df.drop('MachineID', axis=1)
        elif 'ClientItemKey' in system_df.columns:
            system_df = system_df.drop('ClientItemKey', axis=1)
        elif 'created_date' in system_df.columns:
            system_df = system_df.drop('created_date', axis=1)
        
    return system_df

final_df = join_all_features(feature_datasets, df)

In [72]:
# Set the notebook to display all columns of a dataframe
pd.set_option('display.max_columns', None)

final_df.sample(10)

,ItemKey,RWB_EFFECTIVE_DATE,Is_Virtual_Machine0,Client0,Active0,Days Since Creation,Days Since Last Logon,has_powerbi,num_installed_programs,avg_software_age,num_x64addins,num_x86addins,x64add_avgloadtime,x86add_avgloadtime,has_cap_iq_add,has_factset_add,has_bluematrix_add,has_bloomberg_add,has_acrobat_add,Accessx86_addin_filesize,Excelx64_addin_filesize,Excelx86_addin_filesize,OneNotex64_addin_filesize,OneNotex86_addin_filesize,Outlookx64_addin_filesize,Outlookx86_addin_filesize,PowerPointx64_addin_filesize,PowerPointx86_addin_filesize,Publisherx86_addin_filesize,Wordx64_addin_filesize,Wordx86_addin_filesize,CaseType,created_date,num_updates,num_windows_64_os_updates,num_office_updates,Battery_Power,AC_Power,EstimatedRunTime00,MicrosoftEdgeCP_usage,chrome_usage,firefox_usage,iexplore_usage,msedge_usage,opera_usage,Model,system_type,TotalSpace_GB,FreeSpace_GB,is_32bit_office,is_64bit_office,Has32BitMacro,Has64BitMacro,Has32BitOffice97_2003Files,Has64BitOffice97_2003Files,NumCPUs,AvgProcessorNormSpeed,num_users,TotalBootDurationInMilliseconds,events,InstallDate00,LastBootUpTime00,TotalVirtualMemorySize00,BIOS Release Date,Total RAM,Total Page File Space,LastHWScan,LastHWScan_Days
753920,16814125,2023-04-10,0.0,1.0,1.0,186,12.0,0.0,104.0,538.952941,3.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,198024.0,633424.0,NaN,NaN,539272.0,3796752.0,NaN,527784.0,NaN,NaN,572304.0,Laptop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.0,0.0,0.0,0.0,0.0,Latitude 7320,x64-based PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2021-11-03 23:51:07.000,2023-04-04 02:31:32.000,32490.0,2021-09-13 00:00:00.000,16492932.0,16777216.0,2023-04-06 09:31:23.000,4.0
683164,16813002,2023-01-23,0.0,1.0,1.0,182,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,2023-01-23,6.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latitude 9520,x64-based PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2022-07-25 01:59:12.000,2023-01-13 02:00:52.000,32490.0,2022-09-16 00:00:00.000,NaN,NaN,2023-01-13 06:08:19.000,10.0
553850,16811030,2023-01-17,0.0,1.0,1.0,238,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,2023-01-17,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latitude 9520,x64-based PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30059.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
734628,16813943,2023-05-13,0.0,1.0,1.0,220,9.0,0.0,56.0,614.071429,17.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,745032.0,236576.0,NaN,NaN,4485512.0,579904.0,1338200.0,NaN,NaN,1404704.0,38552.0,Laptop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.0,0.0,0.0,0.0,0.0,Latitude 9510,x64-based PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2021-05-26 22:29:26.000,2023-05-10 02:01:15.000,32431.0,2022-09-12 00:00:00.000,NaN,NaN,2023-05-12 14:22:01.000,1.0
644004,16812464,2023-05-06,0.0,1.0,1.0,298,5.0,0.0,64.0,302.319149,8.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,805720.0,1173000.0,NaN,NaN,2161544.0,5073952.0,607696.0,1066928.0,NaN,607696.0,1110880.0,Laptop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.0,0.0,0.0,0.0,0.0,Latitude 9520,x64-based PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,33377.0,2.0,2022-07-11 04:05:39.000,2023-05-05 08:17:02.000,32490.0,2022-06-30 00:00:00.000,16492932.0,16777216.0,2023-05-05 13:34:23.000,1.0
293553,16805538,2023-05-11,1.0,1.0,1.0,666,3.0,0.0,56.0,559.918919,23.0,8.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,30417952.0,794816.0,NaN,NaN,35093256.0,1797000.0,30953928.0,539600.0,NaN,31020432.0,539600.0,Desktop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,100.0,0.0,0.0,0.0,0.0,Precision Tower 3431,x64-based PC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,32584.0,1.0,2021-07-14 17:14:17.000,2023-05-10 02:00:47.000,33569.0,2022-11-11 00:00:00.000,NaN,NaN,2023-05-10 16:14:50.000,1.0
577410,16811477,2023-01-29,0.0,1.0,1.0,229,6.0,0.0,64.0,409.360000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laptop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Latitude 9520,x64-based

### Step 3: Encode Categorical Features and Handle NaNs

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

def encode_attr(in_df, attrs):
    
    # Get the category labels and  categories
    categories = [(attr, list(in_df[attr].unique())) for attr in attrs]
    ohe_columns = [x[0] for x in categories]
    ohe_categories = [x[1] for x in categories]

    # Instantiate the one hot encoder and fit it 
    enc = OneHotEncoder(sparse_output=False, categories=ohe_categories)
    transformer = make_column_transformer((enc, ohe_columns), remainder='passthrough')
    output = transformer.fit_transform(in_df)

    # Put results in dataframe and clean up columns
    out_df = pd.DataFrame(output, columns=transformer.get_feature_names_out())
    remainder_cols =  [col[11:] for col in out_df.columns if 'remainder' in col]
    encoded_cols = [col[15:] for col in out_df.columns if 'onehotencoder' in col]
    out_df.columns = encoded_cols + remainder_cols
    out_df = out_df[remainder_cols + encoded_cols]

    return out_df